### 상관계수

In [ ]:
#판다스로 상관관계 분석
import pandas as pd

df[var_list].corr()

In [ ]:
#pairplot 그리기
import seaborn as sns

sns.pairplot(df[var_list], kind="reg", height=5)

In [ ]:
#scipy로 상관계수 계산 & 검정
import scipy.stats as stats
import seaborn as sns
import matplotlib.pyplot as plt

r = stats.pearsonr(df["y"], df["x"])
sns.regplot(x=df["x"], y=df["y"])
plt.show()
print("상관계수 : {}, p-value : {}".format(r[0], r[1]))

In [ ]:
#상관계수 행렬을 heatmap으로 표현하기
import seaborn as sns
import matplotlib.pyplot as plt

plt.figure(figsize=(12, 10))
plt.rcParams["font.size"] = 14
colormap = plt.cm.PuBu
sns.heatmap(df[var_list].corr(), cmap=colormap, annot = True, annot_kws = {"size" : 16})
plt.show()

### 회귀분석

In [ ]:
#ols(최소자승법) 다중회귀 분석
import statsmodels.formula.api as smf

model = smf.ols(formula="종속변수 ~ 독립변수1 + 독립변수2", data=df)
result = model.fit()
result.summary()
result.aic #AIC 확인
result.rsquared_adj #수정된 r-square 확인
result.pvalues #p-value 확인 (배열로 접근하면 변수별로 확인 가능)

In [ ]:
#다중공선성 확인을 위해 VIF를 구하겠다면 아래와 같이
from statsmodels.stats.outliers_influence import variance_inflation_factor

variance_inflation_factor(model.exog, i)

#VIF가 10이 넘으면 다중공선성 있다고 판단하며 5가 넘으면 주의할 필요가 있는 것으로 봅니다.
#독립 변수 a와 b가 서로 상관 관계가 있다고 했을 때 두 변수 모두 VIF가 높습니다.
#어느 하나만 VIF가 높은 경우는 없습니다.
#박수도 오른손과 왼손이 있어야 칠 수 있듯이 서로 연관 있는 변수끼리 VIF가 높습니다.

In [ ]:
#후진제거법
import statsmodels.formula.api as smf
def backward_elimination(df, dep_var, ind_vars, select_criteria = "AIC", return_case = "selected_vars"):
    alpha = 0.05
    selected_vars = ind_vars.copy()
    eliminated_vars = []
    done = False
    
    while done is False:
        #step1 : 변수 하나씩을 제외한 n개의 모델을 만들어낸다. (n : 변수 갯수)
        model_result_list = []
        for var in ind_vars:
            var_list = ind_vars.copy()
            if len(eliminated_vars)>0:
                var_list.remove(eliminated_vars)
            var_list.remove(var)
            this_ind_vars_str = " + ".join(var_list)
            this_model_fit = smf.ols(dep_var+" ~ "+this_ind_vars_str, data=df).fit()
            this_result = {
                "selected var" : this_ind_vars_str,
                "eliminated var" : var,            
                "AIC" : this_model_fit.aic,
                "rsquared_adj" : this_model_fit.rsquared_adj
            }
            model_result_list.append(this_result)
        result = pd.DataFrame(model_result_list)
        #step2 : n개 모델들 중 가장 좋은 결과를 보인 모델을 선택하고, 제외 후보 변수를 찾는다.
        ascending = True
        if select_criteria == "rsquared_adj":
            ascending = False
        result.sort_values(by=select_criteria, ascending=ascending, inplace=True)
        candidate_var = result.iloc[0, [1]].values[0]
        #step3 : 제외 후보 변수로 fitting하여 p-value를 알아보고 유의하다면 종료한다.
        candidate_model_fit = smf.ols(dep_var+" ~ "+candidate_var, data=df).fit()
        if candidate_model_fit.pvalues[candidate_var] > alpha: #유의하지 않다면
            eliminated_vars.append(candidate_var) #제거확정
            selected_vars.remove(candidate_var)
        else: #유의하다면 종료한다
            done = True
    if return_case == "s":
        print("후보 독립변수들 : {}".format(ind_vars))
        print("후진제거법 선택변수 : {}".format(selected_vars))
        return selected_vars
    elif return_case == "e":
        return eliminated_vars

backward_elimination(df, dep_var, ind_vars, "AIC")

In [ ]:
#전진 선택법 (단계적 선택법도 옵션에 포함)
import statsmodels.formula.api as smf
def forward_selection(df, dep_var, ind_vars, select_criteria = "AIC", stepwise=False):
    if stepwise:
        #후진제거법으로 제거되는 변수를 먼저 찾아 놓는다.
        eliminated_vars = backward_elimination(df, dep_var, ind_vars, "AIC", "e")
    candidate_vars = ind_vars.copy()
    selected_vars = [] #전진선택법으로 선택된 변수들
    alpha = 0.05 #새롭게 편입된 변수의 회귀계수 p-value가 0.05보다 작을 경우에만 선택
    
    while len(candidate_vars) > 0: #후보 변수 리스트가 모두 소거될 때까지 반복한다.
        candidate_vars_reg_result = []
        for candidate in candidate_vars:
            #새로운 변수가 추가되었을 때 회귀모형의 AIC와 r-square값을 구해본다.
            this_ind_vars_str = candidate
            if len(selected_vars) > 0:
                this_ind_vars_str = " + ".join(selected_vars) + " + " + candidate
            this_model_fit = smf.ols(dep_var+" ~ "+this_ind_vars_str, data=df).fit()
            this_result = {
                "var" : candidate,
                "AIC" : this_model_fit.aic,
                "rsquared_adj" : this_model_fit.rsquared_adj,
                "p-value" : this_model_fit.pvalues[candidate]
            }
            candidate_vars_reg_result.append(this_result)
        result = pd.DataFrame(candidate_vars_reg_result)
        ascending = True
        if select_criteria == "rsquared_adj":
            ascending = False
        result.sort_values(by=select_criteria, ascending=ascending, inplace=True)
        selected_var = result.iloc[0,[0]].values[0]
        selected_var_pvalue = result.iloc[0,[3]].values[0]
        if selected_var_pvalue > alpha:
            break
        selected_vars.append(selected_var)
        candidate_vars.remove(selected_var)
        if stepwise:
            #selected_vars 중에 eliminated_var에 해당하는 것이 있다면 제거한다
            if len(selected_vars) > 2 and len(eliminated_vars) > 0:
                for ev in eliminated_vars:
                    if ev in selected_vars:
                        selected_vars.remove(ev)
    print("후보 독립변수들 : {}".format(ind_vars))
    if stepwise:
        print("단계적 선택법 선택변수 : {}".format(selected_vars))
    else:
        print("전진선택법 선택변수 : {}".format(selected_vars))
    return selected_vars

forward_selection(df, dep_var, ind_vars, "AIC")

In [ ]:
#단계적 선택법
forward_selection(df, dep_var, ind_vars, "AIC", stepwise=True)

### 회귀분석 - 잔차의 독립성, 정규성, 등분산성 검증

In [ ]:
#잔차의 독립성 검증

#잔차의 독립성 : 잔차들끼리 상관관계가 없다는 것을 의미. 즉, 자기상관이 없다는 것
#자기상관이 있다면 F값과 t값, r스퀘어 값이 과장되어 나타날 수 있음
#잔차의 독립성이 확보되어야 회귀모형의 검증력이 인정을 받음

#방법 : Durbin-watson 검정으로 체크
#더빈왓슨 수치의 해석 : 0 ~ 4 사이의 값을 가지며 2와 가까울수록 독립성이 있다고 봄.
#0이나 4에 가까운 숫자라면 독립성이 없다고 봄
#summary()에 있는 더빈왓슨 결과값을 확인하면 됨

model_fit = smf.ols("종속변수 ~ 독립변수", data=df).fit()
model_fit.summary()

In [ ]:
# 잔차의 정규성 검증

# 1.샤피로 윌크 검정으로 체크. p-value가 0.05보다 작다면 귀무가설(=정규성이 있다)을 기각
#   예를 들어 p-value가 1.44e-13으로 0.05보다 낮게 나올 경우 정규성이 없다고 판단
from scipy import stats

model_fit = smf.ols("종속변수 ~ 독립변수", data=df).fit()

predict = model_fit.predict(df)
predict.name="predict"

residual = df[dep_var] - predict
residual.name="residual"

stats.shapiro(residual)


# 2. Q-Q plot으로 시각화하여 정규성 체크
#    X축은 정규분포곡선의 확률분포 위치값
#    Y축은 잔차가 정규분포곡선의 분포와 비교해 얼마나 떨어져 있는지의 값
#    붉은선에서 벗어나 있으면(대각선이 아니면) 정규성 없다고 봄

stats.probplot(residual, plot=plt)
plt.title("Q-Q plot")
plt.show()

In [ ]:
# 잔차의 등분산성 검증

#붉은색 선이 직선에 가까우면 등분산으로 볼 수 있음
sns.regplot(x=predict, y=residual, lowess=True, line_kws={'color': 'red'})
plt.plot([predict.min(), predict.max()], [0, 0], '--', color='grey')